In [196]:
from pymongo import MongoClient
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
load_dotenv()

True

In [220]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [221]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [222]:
companies = db.get_collection("companies")

In [223]:
proyec = {"name":1,"category_code": 1, "_id": 0, "offices.city":1,"total_money_raised":1}
cond = {"category_code": "games_video"}
list(companies.find(cond,proyec))[0]


{'name': 'Flektor',
 'category_code': 'games_video',
 'total_money_raised': '$0',
 'offices': [{'city': 'Culver City'}]}

In [224]:
selec = pd.DataFrame(list(companies.find(cond,proyec)))

In [225]:
games = selec[selec["total_money_raised"]!= "$0"]

In [226]:
games

,name,category_code,total_money_raised,offices
1,Lala,games_video,$44.2M,[{'city': 'Palo Alto'}]
2,Joost,games_video,$45M,[{'city': 'New York'}]
3,Babelgum,games_video,$13.2M,[{'city': 'London'}]
5,Kyte,games_video,$23.4M,[{'city': 'San Francisco'}]
6,Veoh,games_video,$69.8M,[{'city': 'San Diego'}]
...,...,...,...,...
1053,MindFuse,games_video,$1M,[]
1056,Ph03nix New Media,games_video,$1.1M,"[{'city': 'Calgary, Alberta'}]"
1069,Eyetronics,games_video,$4.72M,[{'city': ''}]
1070,Antix Labs,games_video,$3M,[{'city': 'Berkshire'}]


In [227]:
#list(selec.total_money_raised.unique())

In [228]:
#\d+[.\d]?

In [229]:
games["amount"] = games.total_money_raised.str.extract(r"(\d+\.?\d?)(.$)")[0]
games["type"] = games.total_money_raised.str.extract(r"(\d+\.?\d?)(.$)")[1]

<ipython-input-229-d3fb5fb4349e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games["amount"] = games.total_money_raised.str.extract(r"(\d+\.?\d?)(.$)")[0]
<ipython-input-229-d3fb5fb4349e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games["type"] = games.total_money_raised.str.extract(r"(\d+\.?\d?)(.$)")[1]


In [230]:
games["amount"] = games["amount"].astype("float")

<ipython-input-230-9a2871b1a742>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games["amount"] = games["amount"].astype("float")


In [231]:
games[games["type"]!= "k"]
 
    
    

,name,category_code,total_money_raised,offices,amount,type
1,Lala,games_video,$44.2M,[{'city': 'Palo Alto'}],44.2,M
2,Joost,games_video,$45M,[{'city': 'New York'}],45.0,M
3,Babelgum,games_video,$13.2M,[{'city': 'London'}],13.2,M
5,Kyte,games_video,$23.4M,[{'city': 'San Francisco'}],23.4,M
6,Veoh,games_video,$69.8M,[{'city': 'San Diego'}],69.8,M
...,...,...,...,...,...,...
1053,MindFuse,games_video,$1M,[],1.0,M
1056,Ph03nix New Media,games_video,$1.1M,"[{'city': 'Calgary, Alberta'}]",1.1,M
1069,Eyetronics,games_video,$4.72M,[{'city': ''}],72.0,M
1070,Antix Labs,games_video,$3M,[{'city': 'Berkshire'}],3.0,M


In [137]:
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {"type": "Point", "coordinates": [data["latt"], data["longt"]]}
    except:
        return data

In [140]:
madrid = geocode("P.º de la Chopera, 6, 28028 Madrid")
madrid

{'type': 'Point', 'coordinates': ['40.39407', '-3.70045']}

In [65]:
madrid = geocode("Madrid")
madrid

{'success': False,
 'error': {'code': '006',
  'message': 'Request Throttled. Over Rate limit: up to 2 per sec. See geocode.xyz/pricing',
  'requests': '5'}}

In [10]:
madrid_porsifalla = {'type': 'Point', 'coordinates': [-3.6793, 40.42955]}

In [141]:
madrid["coordinates"]

['40.39407', '-3.70045']

In [142]:
f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}"

'40.39407, -3.70045'

In [143]:
client_id = os.getenv("four")
client_secret = os.getenv("foursec")

In [144]:

#url = "https://favqs.com/api/session"
url_query = 'https://api.foursquare.com/v2/venues/search'
url_recomendados = 'https://api.foursquare.com/v2/venues/explore'

In [145]:

parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "restaurante"
}

In [146]:
resp = requests.get(url_query, params = parametros).json()

In [147]:
resp["response"]["venues"][0]["location"]["distance"]

640

In [148]:
#resp["response"]["venues"][0]

In [149]:
madrid

{'type': 'Point', 'coordinates': ['40.39407', '-3.70045']}

In [150]:
companies

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

In [99]:
companies.find_one()["total_money_raised"]

'$39.8M'

In [152]:
proyec = {"category_code" : 1, "_id":0}
list(companies.find({},proyec))

[{'category_code': 'web'},
 {'category_code': 'enterprise'},
 {'category_code': 'software'},
 {'category_code': 'news'},
 {'category_code': 'social'},
 {'category_code': 'network_hosting'},
 {'category_code': 'web'},
 {'category_code': 'web'},
 {'category_code': 'games_video'},
 {'category_code': 'web'},
 {'category_code': 'social'},
 {'category_code': 'web'},
 {'category_code': 'web'},
 {'category_code': 'news'},
 {'category_code': 'music'},
 {'category_code': 'games_video'},
 {'category_code': 'mobile'},
 {'category_code': 'web'},
 {'category_code': 'social'},
 {'category_code': 'games_video'},
 {'category_code': 'news'},
 {'category_code': 'web'},
 {'category_code': 'games_video'},
 {'category_code': 'web'},
 {'category_code': 'network_hosting'},
 {'category_code': 'web'},
 {'category_code': 'search'},
 {'category_code': 'advertising'},
 {'category_code': 'messaging'},
 {'category_code': 'advertising'},
 {'category_code': 'advertising'},
 {'category_code': 'web'},
 {'category_code':